In [2]:
%pip install pydicom
%pip install torchmetrics
%pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
%pip install torchvision

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached pydicom-2.3.1-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
  Using cached torchmetrics-0.11.0-py3-none-any.whl (512 kB)
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 446.2 kB/s e

In [4]:
s3 = boto.client('s3')
sagemaker_session = sagemaker.Session()
bucket = 'mammographydata'
key_prefix = 'DataSet/processed/'
download_area = 'all_data' 
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket= bucket, Prefix= key_prefix)
file_num = 0
page_num = 0
for page in pages:
    response = page['Contents']
    response = response[1:]
    for file in response:
        file_num += 1
    #     key = file['Key'].split('/')
    #     patient_ID = key[2]
    #     view = key[3]
    #     download_path = f'{download_area}/{patient_ID}'
    #     specific_prefix = f'{key_prefix}{patient_ID}/{view}'
    #     sagemaker_session.download_data(download_path, bucket, specific_prefix, extra_args=None)
    #     file_num += 1
    #     if file_num % 400 == 0:
    #         print(f'On file num: {file_num}')
    # print(f'On Page: {page_num}')
    # page_num += 1
print(file_num)

15225


In [7]:
s3 = boto.client('s3')
sagemaker_session = sagemaker.Session()
bucket = 'mammographydata'
key_prefix = 'DataSet/dv/'
download_area = 'dv_data' 
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket= bucket, Prefix= key_prefix)
file_num = 0
page_num = 0
for page in pages:
    response = page['Contents']
    response = response[1:]
    for file in response:
        file_num += 1
        key = file['Key'].split('/')
        patient_ID = key[2]
        view = key[3]
        download_path = f'{download_area}/{patient_ID}'
        specific_prefix = f'{key_prefix}{patient_ID}/{view}'
        sagemaker_session.download_data(download_path, bucket, specific_prefix, extra_args=None)
        file_num += 1
        if file_num % 400 == 0:
            print(f'On file num: {file_num}')
    print(f'On Page: {page_num}')
    page_num += 1

On Page: 0
104


In [11]:
s3 = boto.client('s3')
sagemaker_session = sagemaker.Session()
bucket = 'mammographydata'
key_prefix = 'DataSet/processed/'
download_area = 'all_data' 
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket= bucket, Prefix= key_prefix)
i = 0
j = 0
labels = pickle.load(open('label_dict.pt', 'rb'))
undownloaded_list = []
for label in labels.keys():
    if label not in os.listdir('all_data'):
        i += 1
    files = os.listdir(f'all_data/{label}')
    if  'LCC.pt' not in files or 'LMLO.pt' not in files or 'RCC_flipped.pt' not in files or 'RMLO_flipped.pt' not in files:
        print(label)
        j += 1
        undownloaded_list.append(label)
print(f'NUMBER OF UNDOWNLOADED DATA: {i}')
print(f'NUMBER OF ISSUES: {j}')

NUMBER OF UNDOWNLOADED DATA: 0
NUMBER OF ISSUES: 0


In [10]:
s3 = boto.client('s3')
sagemaker_session = sagemaker.Session()
bucket = 'mammographydata'
key_prefix = 'DataSet/processed/'
download_area = 'all_data' 
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket= bucket, Prefix= key_prefix)
file_num = 0
page_num = 0
views = ['LCC.pt','LMLO.pt','RCC_flipped.pt','RMLO_flipped.pt']
# for page in pages:
#     response = page['Contents']
#     response = response[1:]
for file in undownloaded_list:
    # file_num += 1
    # key = file['Key'].split('/')
    patient_ID = file
    # view = key[3]
    download_path = f'{download_area}/{patient_ID}'
    for view in views:
        specific_prefix = f'{key_prefix}{patient_ID}/{view}'
        sagemaker_session.download_data(download_path, bucket, specific_prefix, extra_args=None)
    # file_num += 1
    # if file_num % 400 == 0:
        # print(f'On file num: {file_num}')
# print(f'On Page: {page_num}')
# page_num += 1
print(file_num)

0


In [2]:
import boto3 as boto
import pickle
import torch
import sagemaker
import pandas as pd
import os
import numpy as np
import pydicom as dicom
from torch import nn

In [3]:
from mammography_project.src.models import data
from mammography_project.src.models import MyClasses
from mammography_project.src.models import BatchedMyClasses
from mammography_project.src.models import PaperModel
from mammography_project.src.models import runners

In [ ]:
PaperModel.main(batch_size = 2, device = 'cuda')

LEN TEST GEN BEFORE: 762
LEN TRAIN GEN BEFORE: 3048
LEN TEST GEN: 762
LEN TRAIN GEN: 3048
Starting mean_and_variance
Finishing mean_and_variance
Starting mean_and_variance
Finishing mean_and_variance
Trying Batched
Starting Training
	[GPU cuda] Epoch 1
Have run 20 batches
Have run 40 batches
Have run 60 batches
Have run 80 batches
Have run 100 batches
Have run 120 batches
Have run 140 batches
Have run 160 batches
Have run 180 batches
Have run 200 batches
Have run 220 batches
Have run 240 batches
Have run 260 batches
Have run 580 batches
Have run 760 batches
Have run 860 batches
Have run 880 batches
Have run 900 batches
